# Homework 6
The goal of this homework is to create a regression model for predicting the car fuel efficiency (column 'fuel_efficiency_mpg').

In [1]:
!wget https://raw.githubusercontent.com/alexeygrigorev/datasets/master/car_fuel_efficiency.csv

--2025-11-04 04:36:45--  https://raw.githubusercontent.com/alexeygrigorev/datasets/master/car_fuel_efficiency.csv
Loaded CA certificate '/etc/ssl/certs/ca-certificates.crt'
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 2606:50c0:8002::154, 2606:50c0:8003::154, 2606:50c0:8000::154, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|2606:50c0:8002::154|:443... connected.
HTTP request sent, awaiting response... 416 Range Not Satisfiable

    The file is already fully retrieved; nothing to do.



## Data Preparation
Preparation:

- Fill missing values with zeros.
- Do train/validation/test split with 60%/20%/20% distribution.
- Use the train_test_split function and set the random_state parameter to 1.
- Use DictVectorizer(sparse=True) to turn the dataframes into matrices.

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
df = pd.read_csv('car_fuel_efficiency.csv')
df

,engine_displacement,num_cylinders,horsepower,vehicle_weight,acceleration,model_year,origin,fuel_type,drivetrain,num_doors,fuel_efficiency_mpg
0,170,3.0,159.0,3413.433759,17.7,2003,Europe,Gasoline,All-wheel drive,0.0,13.231729
1,130,5.0,97.0,3149.664934,17.8,2007,USA,Gasoline,Front-wheel drive,0.0,13.688217
2,170,NaN,78.0,3079.038997,15.1,2018,Europe,Gasoline,Front-wheel drive,0.0,14.246341
3,220,4.0,NaN,2542.392402,20.2,2009,USA,Diesel,All-wheel drive,2.0,16.912736
4,210,1.0,140.0,3460.870990,14.4,2009,Europe,Gasoline,All-wheel drive,2.0,12.488369
...,...,...,...,...,...,...,...,...,...,...,...
9699,140,5.0,164.0,2981.107371,17.3,2013,Europe,Diesel,Front-wheel drive,NaN,15.101802
9700,180,NaN,154.0,2439.525729,15.0,2004,USA,Gasoline,All-wheel drive,0.0,17.962326
9701,220,2.0,138.0,2583.471318,15.1,2008,USA,Diesel,All-wheel drive,-1.0,17.186587
9702,230,4.0,177.0,2905.527390,19.4,2011,USA,Diesel,Front-wheel drive,1.0,15.331551


### Fill missing values with zeros.

In [4]:
df.describe()

,engine_displacement,num_cylinders,horsepower,vehicle_weight,acceleration,model_year,num_doors,fuel_efficiency_mpg
count,9704.000000,9222.000000,8996.000000,9704.000000,8774.000000,9704.000000,9202.000000,9704.000000
mean,199.708368,3.962481,149.657292,3001.280993,15.021928,2011.484027,-0.006412,14.985243
std,49.455319,1.999323,29.879555,497.894860,2.510339,6.659808,1.048162,2.556468
min,10.000000,0.000000,37.000000,952.681761,6.000000,2000.000000,-4.000000,6.200971
25%,170.000000,3.000000,130.000000,2666.248985,13.300000,2006.000000,-1.000000,13.267459
50%,200.000000,4.000000,149.000000,2993.226296,15.000000,2012.000000,0.000000,15.006037
75%,230.000000,5.000000,170.000000,3334.957039,16.700000,2017.000000,1.000000,16.707965
max,380.000000,13.000000,271.000000,4739.077089,24.300000,2023.000000,4.000000,25.967222


In [5]:
df.dtypes

engine_displacement      int64
num_cylinders          float64
horsepower             float64
vehicle_weight         float64
acceleration           float64
model_year               int64
origin                  object
fuel_type               object
drivetrain              object
num_doors              float64
fuel_efficiency_mpg    float64
dtype: object

In [6]:
df.isnull().sum()

engine_displacement      0
num_cylinders          482
horsepower             708
vehicle_weight           0
acceleration           930
model_year               0
origin                   0
fuel_type                0
drivetrain               0
num_doors              502
fuel_efficiency_mpg      0
dtype: int64

In [7]:
df_prep = df.copy()
for col in df_prep.columns:
    if df_prep[col].dtype == 'object':
        df_prep[col] = df_prep[col].fillna('NA')
    else:
        df_prep[col] = df_prep[col].fillna(0.0)

df_prep.isnull().sum()

engine_displacement    0
num_cylinders          0
horsepower             0
vehicle_weight         0
acceleration           0
model_year             0
origin                 0
fuel_type              0
drivetrain             0
num_doors              0
fuel_efficiency_mpg    0
dtype: int64

### Do train/validation/test split with 60%/20%/20% distribution.

Use the train_test_split function and set the random_state parameter to 1.

In [8]:
from sklearn.model_selection import train_test_split


df_full_train, df_test = train_test_split(df_prep, test_size=0.2, random_state=1)
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=1)

df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)


In [9]:
from sklearn.feature_extraction import DictVectorizer
dv = DictVectorizer(sparse=True)

## Question 1

Let's train a decision tree regressor to predict the `fuel_efficiency_mpg` variable. 

* Train a model with `max_depth=1`.


Which feature is used for splitting the data?


* `'vehicle_weight'`
* 'model_year'
* 'origin'
* 'fuel_type'


In [10]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_extraction import DictVectorizer
from sklearn.metrics import roc_auc_score
from sklearn.tree import export_text

In [ ]:
from sklearn.tree import DecisionTreeRegressor

features = [c for c in df_train.columns if c != col]
y_train = df_train[col].values

train_dict = df_train[features].to_dict(orient='records')
X_train = dv.fit_transform(train_dict)

dt = DecisionTreeRegressor(max_depth=1)
dt.fit(X_train, y_train)

try:
    feature_names = dv.get_feature_names_out()
except AttributeError:
    feature_names = dv.get_feature_names()

split_idx = dt.tree_.feature[0]
print("split feature:", feature_names[split_idx])

split feature: vehicle_weight


## Question 2

Train a random forest regressor with these parameters:

* `n_estimators=10`
* `random_state=1`
* `n_jobs=-1` (optional - to make training faster)


What's the RMSE of this model on the validation data?

* 0.045
* `0.45`
* 4.5
* 45.0

In [12]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(n_estimators=10, random_state=1, n_jobs=-1)
rf.fit(X_train, y_train)

val_dict = df_val[features].to_dict(orient='records')
X_val = dv.transform(val_dict)
y_val = df_val[col].values

y_pred = rf.predict(X_val)
rmse = np.sqrt(((y_val - y_pred) ** 2).mean())
print("RMSE:", rmse)

RMSE: 0.4595777223092726


## Question 3

Now let's experiment with the `n_estimators` parameter

* Try different values of this parameter from 10 to 200 with step 10.
* Set `random_state` to `1`.
* Evaluate the model on the validation dataset.


After which value of `n_estimators` does RMSE stop improving?
Consider 3 decimal places for calculating the answer.

- 10
- 25
- 80
- `200`

If it doesn't stop improving, use the latest iteration number in
your answer.

In [ ]:
ns = list(range(10, 201, 10))

train_dict = df_train[features].to_dict(orient='records')
X_train = dv.transform(train_dict)

val_dict = df_val[features].to_dict(orient='records')
X_val = dv.transform(val_dict)

y_val = df_val['fuel_efficiency_mpg'].values

rmses = []
for n in ns:
    rf = RandomForestRegressor(n_estimators=n, random_state=1, n_jobs=-1)
    rf.fit(X_train, y_train)
    y_pred = rf.predict(X_val)
    rmse = np.sqrt(((y_val - y_pred) ** 2).mean())
    rmses.append(rmse)

rmses_3 = [round(r, 3) for r in rmses]

stop_n = ns[-1]
for i in range(len(ns) - 1):
    current = rmses_3[i]
    later_min = min(rmses_3[i+1:])
    if later_min >= current:
        stop_n = ns[i]
        break

# print results
for n, r in zip(ns, rmses_3):
    print(f"n_estimators={n:3d}  RMSE={r:.3f}")
print()
print(f"RMSE stops improving (3-decimal) after n_estimators = {stop_n}")

n_estimators= 10  RMSE=0.460
n_estimators= 20  RMSE=0.454
n_estimators= 30  RMSE=0.452
n_estimators= 40  RMSE=0.449
n_estimators= 50  RMSE=0.447
n_estimators= 60  RMSE=0.445
n_estimators= 70  RMSE=0.445
n_estimators= 80  RMSE=0.445
n_estimators= 90  RMSE=0.445
n_estimators=100  RMSE=0.445
n_estimators=110  RMSE=0.444
n_estimators=120  RMSE=0.444
n_estimators=130  RMSE=0.444
n_estimators=140  RMSE=0.443
n_estimators=150  RMSE=0.443
n_estimators=160  RMSE=0.443
n_estimators=170  RMSE=0.443
n_estimators=180  RMSE=0.442
n_estimators=190  RMSE=0.442
n_estimators=200  RMSE=0.442

RMSE stops improving (3-decimal) after n_estimators = 180


## Question 4

Let's select the best `max_depth`:

* Try different values of `max_depth`: `[10, 15, 20, 25]`
* For each of these values,
  * try different values of `n_estimators` from 10 till 200 (with step 10)
  * calculate the mean RMSE 
* Fix the random seed: `random_state=1`


What's the best `max_depth`, using the mean RMSE?

* `10`
* 15
* 20
* 25

In [ ]:
from sklearn.ensemble import RandomForestRegressor
import numpy as np

max_depths = [10, 15, 20, 25]
ns = list(range(10, 201, 10))

results = {}
for md in max_depths:
    rmses = []
    for n in ns:
        rf = RandomForestRegressor(n_estimators=n, max_depth=md, random_state=1, n_jobs=-1)
        rf.fit(X_train, y_train)
        y_pred = rf.predict(X_val)
        rmse = np.sqrt(((y_val - y_pred) ** 2).mean())
        rmses.append(rmse)
    mean_rmse = float(np.mean(rmses))
    results[md] = mean_rmse
    print(f"max_depth={md}  mean_RMSE={mean_rmse:.3f}")

best_md = min(results, key=results.get)
print()
print(f"Best max_depth (by mean RMSE): {best_md} (mean RMSE={results[best_md]:.3f})")

max_depth=10  mean_RMSE=0.442
max_depth=15  mean_RMSE=0.445
max_depth=20  mean_RMSE=0.446
max_depth=25  mean_RMSE=0.446

Best max_depth (by mean RMSE): 10 (mean RMSE=0.442)


# Question 5

We can extract feature importance information from tree-based models. 

At each step of the decision tree learning algorithm, it finds the best split. 
When doing it, we can calculate "gain" - the reduction in impurity before and after the split. 
This gain is quite useful in understanding what are the important features for tree-based models.

In Scikit-Learn, tree-based models contain this information in the
[`feature_importances_`](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html#sklearn.ensemble.RandomForestRegressor.feature_importances_)
field. 

For this homework question, we'll find the most important feature:

* Train the model with these parameters:
  * `n_estimators=10`,
  * `max_depth=20`,
  * `random_state=1`,
  * `n_jobs=-1` (optional)
* Get the feature importance information from this model


What's the most important feature (among these 4)? 

* `vehicle_weight`
*	`horsepower`
* `acceleration`
* `engine_displacement`	

In [ ]:
rf2 = RandomForestRegressor(n_estimators=10, max_depth=20, random_state=1, n_jobs=-1)
rf2.fit(X_train, y_train)

importances = rf2.feature_importances_
fi = dict(zip(feature_names, importances))

candidates = ['vehicle_weight', 'horsepower', 'acceleration', 'engine_displacement']

for f in candidates:
    print(f"{f:20s} importance = {fi.get(f, 0.0):.6f}")

best = max(candidates, key=lambda f: fi.get(f, 0.0))
print("\nMost important feature among the four:", best)

vehicle_weight       importance = 0.959150
horsepower           importance = 0.015998
acceleration         importance = 0.011480
engine_displacement  importance = 0.003273

Most important feature among the four: vehicle_weight


## Question 6

Now let's train an XGBoost model! For this question, we'll tune the `eta` parameter:

* Install XGBoost
* Create DMatrix for train and validation
* Create a watchlist
* Train a model with these parameters for 100 rounds:

```
xgb_params = {
    'eta': 0.3, 
    'max_depth': 6,
    'min_child_weight': 1,
    
    'objective': 'reg:squarederror',
    'nthread': 8,
    
    'seed': 1,
    'verbosity': 1,
}
```

Now change `eta` from `0.3` to `0.1`.

Which eta leads to the best RMSE score on the validation dataset?

* 0.3
* `0.1`
* Both give equal value

In [19]:
%pip install xgboost

  Using cached nvidia_nccl_cu12-2.28.7-py3-none-manylinux_2_18_x86_64.whl.metadata (2.0 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.9/115.9 MB 1.2 MB/s  0:01:37m0:00:0100:03
Using cached nvidia_nccl_cu12-2.28.7-py3-none-manylinux_2_18_x86_64.whl (296.8 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [xgboost]m1/2 [xgboost]
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import xgboost as xgb

dtrain = xgb.DMatrix(X_train, label=y_train)
dval = xgb.DMatrix(X_val, label=y_val)

base_params = {
    'max_depth': 6,
    'min_child_weight': 1,
    'objective': 'reg:squarederror',
    'nthread': 8,
    'seed': 1,
    'verbosity': 1,
    'eval_metric': 'rmse',
}

results = {}
for eta in (0.3, 0.1):
    params = base_params.copy()
    params['eta'] = eta
    evals_result = {}
    model = xgb.train(params, dtrain, num_boost_round=100,
                      evals=[(dtrain, 'train'), (dval, 'val')],
                      evals_result=evals_result, verbose_eval=False)
    rmse_val = evals_result['val']['rmse'][-1]
    results[eta] = rmse_val
    print(f"eta={eta}  RMSE (val) = {rmse_val:.6f}")


if results[0.3] < results[0.1]:
    print("\nBest eta: 0.3")
elif results[0.1] < results[0.3]:
    print("\nBest eta: 0.1")
else:
    print("\nBoth give equal value")

eta=0.3  RMSE (val) = 0.450178
eta=0.1  RMSE (val) = 0.426228

Best eta: 0.1
